In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
from langchain_core.messages.tool import tool_call

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [10]:
from crypto.statistics.CryptoSeries import CryptoSeries
from crypto.dto.types import CryptoInfoRequest
from langchain.agents import tool
from datetime import datetime

@tool(args_schema=CryptoInfoRequest)
def crypto_correlation(symbols: list, start_date: datetime, end_date: datetime):
    """Returns a correlation matrix for the given list of symbols and between start_date and end_date"""
    return CryptoSeries.get().correlation(symbols, start_date, end_date)

crypto_correlation

StructuredTool(name='crypto_correlation', description='Returns a correlation matrix for the given list of symbols and between start_date and end_date', args_schema=<class 'crypto.dto.types.CryptoInfoRequest'>, func=<function crypto_correlation at 0x793bf7f3edd0>)

In [11]:
from langchain_openai import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function

functions = [
    format_tool_to_openai_function(f) for f in [
        crypto_correlation
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

functions

[{'name': 'crypto_correlation',
  'description': 'Returns a correlation matrix for the given list of symbols and between start_date and end_date',
  'parameters': {'properties': {'start_date': {'description': 'Start date timestamp in format: yyyy-MM-ddTHH:mm:ssZ',
     'format': 'date-time',
     'type': 'string'},
    'end_date': {'description': 'End date timestamp in format: yyyy-MM-ddTHH:mm:ssZ',
     'format': 'date-time',
     'type': 'string'},
    'symbols': {'description': 'List of crypto symbols like BTC, ETH',
     'items': {'type': 'string'},
     'type': 'array'}},
   'required': ['start_date', 'end_date', 'symbols'],
   'type': 'object'}}]

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant with crypto knowledge"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
chain = prompt | model

chain.invoke({
    "chat_history": [],
    "input": "what is the best crypto to invest?",
    "agent_scratchpad": []})

AIMessage(content="I'm here to provide information and guidance, but I can't give financial advice or predict the future. The best crypto to invest in depends on various factors such as your investment goals, risk tolerance, and market research. It's important to do your own research, consider the potential risks and rewards, and make informed decisions based on your own financial situation. If you're looking for specific information or analysis on cryptocurrencies, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 143, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3666a67-dad4-43d0-8d24-26f97aed19c5-0', usage_

In [16]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

result = chain.invoke({
    "chat_history": [],
    "input": "what is the correlation between bitcoin and ethereum last year?",
    "agent_scratchpad": []
})

print(f"{result.tool}  {result.tool_input}")

crypto_correlation  {'start_date': '2021-01-01T00:00:00Z', 'end_date': '2021-12-31T23:59:59Z', 'symbols': ['BTC', 'ETH']}


In [26]:
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.memory import ConversationBufferMemory


agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

def run_agent(user_input, memory):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "chat_history": memory,
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool_to_run = {
            "crypto_correlation": crypto_correlation
        }[result.tool]
        observation = tool_to_run.run(result.tool_input)
        intermediate_steps.append((result, observation))


memory = []
msg = "what is the correlation between bitcoin and ethereum last year?"
reply = run_agent(msg, memory)
reply = reply.return_values['output']
print(reply)
memory += [msg, reply]

msg = "my name is bob"
reply = run_agent(msg, memory)
reply = reply.return_values['output']
print(reply)
memory += [msg, reply]

msg = "what is my name and correlations for btc and eth you computed?"
reply = run_agent(msg, memory)
reply = reply.return_values['output']
print(reply)
memory += [msg, reply]

The correlation between Bitcoin (BTC) and Ethereum (ETH) last year was approximately 0.76. This indicates a strong positive correlation between the two cryptocurrencies during that period.
Hello Bob! How can I assist you today?
Your name is Bob. 

The correlation between Bitcoin (BTC) and Ethereum (ETH) that I computed is approximately 0.76. This indicates a strong positive correlation between the two cryptocurrencies during the specified time period.
